# Simple Cycle Examples with Uncertainty vs. Random Experimentalist
The aim of this example notebook is to use the AutoRA `Cycle` to recover a ground truth theory from some noisy data using BSM.
It comparse the default "random" experimentalist with the "uncertainty" sampler.

In [ ]:
# Uncomment the following line when running on Google Colab
# !pip install autora

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.dummy import DummyRegressor

from autora.cycle import Cycle
from autora.experimentalist.sampler import random_sampler, poppernet_pooler, nearest_values_sampler
from autora.experimentalist.pipeline import make_pipeline
from autora.variable import VariableCollection, Variable

In [ ]:
def ground_truth(xs):
    oscillating_component = np.sin((4. * xs) - 3.)
    parabolic_component =  (-0.1 * xs ** 2.) + (2.5 * xs) + 1.
    ys = oscillating_component + parabolic_component
    return ys

The space of allowed x values is reals between -10 and 10 inclusive. We discretize them as we don't currently have a sampler which can sample from the uniform distribution.

In [ ]:
study_metadata = VariableCollection(
   independent_variables=[Variable(name="x1", allowed_values=np.linspace(-10, 10, 500))],
   dependent_variables=[Variable(name="y")],
   )

So that we can compare the effectiveness of the two strategies, we fix the number of observations per cycle to be 100.

In [ ]:
observations_per_cycle = 100

When we run a synthetic experiment, we get a reproducible noisy result:

In [ ]:
import numpy as np

def get_example_synthetic_experiment_runner():
    rng = np.random.default_rng(seed=180)
    def runner(xs):
        return ground_truth(xs) + rng.normal(0, 1.0, xs.shape)
    return runner

example_synthetic_experiment_runner = get_example_synthetic_experiment_runner()
x = np.array([1.])
example_synthetic_experiment_runner(x)

In [ ]:
plt.scatter(study_metadata.independent_variables[0].allowed_values[::5,], example_synthetic_experiment_runner(study_metadata.independent_variables[0].allowed_values[::5,]), alpha=1, s=0.1, c='r', label="samples")
plt.plot(study_metadata.independent_variables[0].allowed_values, ground_truth(study_metadata.independent_variables[0].allowed_values), c="black", label="ground truth")
plt.legend()

We use a common BMS regressor with a common parametrization to test the two methods.

In [ ]:
from autora.skl.bms import BMSRegressor
bms_theorist = BMSRegressor(epochs=100)

We also define a helper function to plot the results

In [ ]:
def run_and_plot_cycle(cycle, study_metadata):
    cycle.run(num_cycles=1)

    all_obs = np.row_stack(cycle.data.observations)
    x_obs, y_obs = all_obs[:,0], all_obs[:,1]
    x_obs_new, y_obs_new = cycle.data.observations[-1][:,0], cycle.data.observations[-1][:,1]

    x_pred = np.array(study_metadata.independent_variables[0].allowed_values).reshape(-1, 1)
    y_pred = cycle.data.theories[-1].predict(x_pred)

    plt.plot(study_metadata.independent_variables[0].allowed_values, ground_truth(study_metadata.independent_variables[0].allowed_values), c="black", label="ground truth")
    plt.scatter(x_obs, y_obs, s=1, c='r', label="samples")
    plt.scatter(x_obs_new, y_obs_new, s=1, c='green', facecolors="none", label="new samples")
    plt.plot(x_pred, y_pred, c="blue", label="theorist result")

    plt.legend()

    plt.show()

## Random Sampler

In [ ]:
random_experimentalist = make_pipeline(
    [study_metadata.independent_variables[0].allowed_values, random_sampler],
    params={"random_sampler": {"n": observations_per_cycle}}
)
random_experimentalist_cycle = Cycle(
    metadata=study_metadata,
    theorist=bms_theorist,
    experimentalist=random_experimentalist,
    experiment_runner=example_synthetic_experiment_runner
)

for _ in range(10):
    run_and_plot_cycle(cycle=random_experimentalist_cycle, study_metadata=study_metadata)

## Popper Sampler

In [ ]:
poppernet_experimentalist = make_pipeline(
    [poppernet_pooler, nearest_values_sampler],
)

poppernet_experimentalist_cycle = Cycle(
    metadata=study_metadata,
    theorist=bms_theorist,
    experimentalist=poppernet_experimentalist,
    experiment_runner=example_synthetic_experiment_runner,
    params={"experimentalist" : {
                "poppernet_pooler": {
                    "model": "%theories[-1]%",
                    "x_train": "%observations.ivs%",
                    "y_train": "%observations.dvs%",
                    "metadata": study_metadata,
                    "num_samples": observations_per_cycle,
                },
                "nearest_values_sampler": {
                    "allowed_values": study_metadata.independent_variables[0].allowed_values
                }
            }
        }
    )

The Popper sampler depends on having a first guess for the theory, so we add an appropriate model and an initial datapoint to the cycle's data:

In [ ]:
# Experimentalist
x_seed = np.linspace(-10, 10, 20)

# Experiment runner
y_seed = example_synthetic_experiment_runner(x_seed)
poppernet_experimentalist_cycle.data.observations.append(np.column_stack([x_seed, y_seed]))

# Theorist
theory_seed = DummyRegressor(strategy="constant", constant=y_seed[1])
theory_seed.fit(x_seed, y_seed)
poppernet_experimentalist_cycle.data.theories.append(theory_seed)

Now we can run the cycle and check the results.

In [ ]:
for _ in range(10):
    run_and_plot_cycle(cycle=poppernet_experimentalist_cycle, study_metadata=study_metadata)
